In [1]:
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import cv2
import tensorflow as tf

In [2]:
# Função para carregar imagens e extrair características HOG
def load_data(data_dir, img_size=(128, 128)):
    labels = []
    features = []
    for label in ['acne', 'no_acne']:
        folder = os.path.join(data_dir, label)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            img = imread(img_path)
            img_resized = resize(img, img_size)
            hog_features = hog(img_resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False, channel_axis=-1)
            features.append(hog_features)
            labels.append(0 if label == 'no_acne' else 1)
    return np.array(features), np.array(labels)

# Carregar dados
data_dir = r'c:\Users\SETH\Desktop\GitHub\modelo_ml_tcc'
X, y = load_data(data_dir)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar um classificador SVM
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Avaliar o modelo
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Função para fazer predições em novas imagens
def predict_image(image_path, model, img_size=(128, 128)):
    img = imread(image_path)
    img_resized = resize(img, img_size)
    hog_features = hog(img_resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False, channel_axis=-1)
    hog_features = np.array(hog_features).reshape(1, -1)
    prediction = model.predict(hog_features)
    if prediction[0] == 1:
        print("Espinha detectada")
    else:
        print("Sem espinha")

Accuracy: 99.77%


In [3]:
predict_image(r'c:\Users\SETH\Desktop\GitHub\modelo_ml_tcc\predict_img\rosto_com_cne.jpg', clf)

Espinha detectada


In [4]:
predict_image(r'c:\Users\SETH\Desktop\GitHub\modelo_ml_tcc\predict_img\capa-3.webp', clf)

Sem espinha


In [22]:
predict_image(r'c:\Users\SETH\Desktop\GitHub\modelo_ml_tcc\predict_img\rosto_1_acne.jpg', clf)

Espinha detectada


In [5]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

In [6]:
# Função para carregar imagens e extrair características HOG
def load_data(data_dir, img_size=(128, 128)):
    labels = []
    features = []
    for label in ['acne', 'no_acne']:
        folder = os.path.join(data_dir, label)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            img = imread(img_path)
            img_resized = resize(img, img_size)
            hog_features = hog(img_resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False, channel_axis=-1)
            features.append(hog_features)
            labels.append(0 if label == 'no_acne' else 1)
    return np.array(features), np.array(labels)

# Carregar dados
data_dir = r'c:\Users\SETH\Desktop\GitHub\modelo_ml_tcc'
X, y = load_data(data_dir)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar um classificador SVM
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Avaliar o modelo
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Salvar o modelo
model_path = 'svm_model.pkl'
joblib.dump(clf, model_path)
print(f'Modelo salvo em {model_path}')

Accuracy: 99.77%
Modelo salvo em svm_model.pkl


In [7]:
# Função para fazer predições em novas imagens
def predict_image(image_path, model_path, img_size=(128, 128)):
    # Carregar o modelo salvo
    model = joblib.load(model_path)
    
    # Processar a imagem
    img = imread(image_path)
    img_resized = resize(img, img_size)
    hog_features = hog(img_resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False, channel_axis=-1)
    hog_features = np.array(hog_features).reshape(1, -1)
    
    # Fazer predição
    prediction = model.predict(hog_features)
    if prediction[0] == 1:
        print("Espinha detectada")
    else:
        print("Sem espinha")

# Exemplo de uso da função predict_image
image_path = r'c:\Users\SETH\Desktop\GitHub\modelo_ml_tcc\predict_img\capa-3.webp'
model_path = 'svm_model.pkl'
predict_image(image_path, model_path)

Sem espinha


In [9]:
import boto3

# Configurar a sessão padrão do boto3
boto3.setup_default_session(profile_name='faculdade')

# Função para fazer upload do arquivo para S3
def upload_to_s3(file_path, bucket_name, s3_file_key):
    s3 = boto3.client('s3')
    s3.upload_file(file_path, bucket_name, s3_file_key)
    print(f'Arquivo {file_path} carregado com sucesso para {bucket_name}/{s3_file_key}')


# Parâmetros para o upload
file_path = './svm_model.pkl'  # Substitua pelo caminho do seu arquivo
bucket_name = 'tcc-dev-saved-model-bucket'         # Substitua pelo nome do seu bucket S3
s3_file_key = 'svm_model.pkl'  # Substitua pelo caminho desejado no bucket S3

# Fazer upload do arquivo para o S3
upload_to_s3(file_path, bucket_name, s3_file_key)


Arquivo ./svm_model.pkl carregado com sucesso para tcc-dev-saved-model-bucket/svm_model.pkl
